In [ ]:
import os
import glob
import pandas as pd
import sys
import numpy as np
from functools import reduce

os.environ["HOME"]

In [ ]:
#Import sample.pVR_freq files as the form of table set
folder_pVR = '.pVR_freq_FILE_PATH'
pVR_files = glob.glob(os.path.join(folder_pVR, "*.pVR_freq"))
pVR_tableset = []
for filename in pVR_files:
    df_pVR = pd.read_csv(filename, sep = '\t')
    pVR_tableset.append(df_pVR)

In [ ]:
#Pair each sample with the sample's table set
new_pVR_f = []
for valname in pVR_files:
    spliced_name = valname.split('/')[-1].split('.')[0]
    new_pVR_f.append(spliced_name)
dict_pVR_df = dict(zip(new_pVR_f, pVR_tableset))

In [ ]:
#Create the dataframe with only the sequence and frequency of dataframes
for key, val in dict_pVR_df.items():
    val.drop(['Rank', 'Count', 'nseq_count', 'pct_over3', 'nseq_rep', 'nseq_rep_count'], axis = 1, inplace = True)
    val.columns = ['Pep_seq', str(key)]

In [ ]:
#Assign the variables with each sample's dataframe : each name of merged columns = dict_pVR_df[str(key)]
Chip_1 = dict_pVR_df['chip_1']
Chip_2 = dict_pVR_df['chip_2']
Chip_3 = dict_pVR_df['chip_3']
Well_1 = dict_pVR_df['well_1']
Well_2 = dict_pVR_df['well_2']
Well_3 = dict_pVR_df['well_3']
Input1_1 = dict_pVR_df['input_1']
Input1_2 = dict_pVR_df['input_2']
Input1_3 = dict_pVR_df['input_3']
Input2_1 = dict_pVR_df['input_new_1']
Input2_2 = dict_pVR_df['input_new_2']
Input2_3 = dict_pVR_df['input_new_3']
Mouse_1 = dict_pVR_df['invivo_1']
Mouse_2 = dict_pVR_df['invivo_2']
Mouse_3 = dict_pVR_df['invivo_3']

In [ ]:
#Change the sequence count frequency to the count per million (CPM)
merge_all = [Input1_1, Input1_2, Input1_3, Input2_1, Input2_2, Input2_3, Well_1, Well_2, Well_3, Chip_1, Chip_2, Chip_3, Mouse_1, Mouse_2, Mouse_3]
merged_pct = reduce(lambda left, right : pd.merge(left, right, on = ['Pep_seq'], how = 'outer'), merge_all)
merged_pct = merged_pct.fillna(0)
merged_pct.columns = ["Pep_seq", "Input1_1", "Input1_2", "Input1_3", "Input2_1", "Input2_2", "Input2_3", "Well_1", "Well_2", "Well_3", "Chip_1", "Chip_2", "Chip_3", "Invivo_1", "Invivo_2", "Invivo_3"]
droped_pct = merged_pct.drop(["Pep_seq"], axis = 1)
merged_pep = merged_pct.drop(["Input1_1", "Input1_2", "Input1_3", "Input2_1", "Input2_2", "Input2_3", "Well_1", "Well_2", "Well_3", "Chip_1", "Chip_2", "Chip_3", "Invivo_1", "Invivo_2", "Invivo_3"], axis = 1)
merged_cpm = droped_pct.mul(10000)
CPM_total = pd.concat([merged_pep, merged_cpm], axis = 1)

In [ ]:
#Remove the sequences with stop codon & outlier sequences (if 0 appears in more than half of the sample columns, it is designated as an outlier and filtered out.)
#t : in vitro
#m : in vivo
CPM_total = CPM_total[~CPM_total['Pep_seq'].str.contains("\*")]
filtered_t = CPM_total.copy()
filtered_m = CPM_total.copy()
#Filtering for in vitro samples (with input lot used for in vitro experiment)
filtered_t = filtered_t.drop(['Input2_1', 'Input2_2', 'Input2_3', 'Invivo_1', 'Invivo_2', 'Invivo_3'], axis = 1)
filtered_t['Count'] = (filtered_t[['Input1_1', 'Input1_2', 'Input1_3', 'Well_1', 'Well_2', 'Well_3', 'Chip_1', 'Chip_2', 'Chip_3']] == 0).sum(axis=1)
filtered_t = filtered_t[filtered_t['Count'] < 6]
filtered_t = filtered_t.drop(['Count'], axis = 1)
#Filtering for in vivo samples (with input lot used for in vivo experiment)
filtered_m = filtered_m.drop(['Input1_1', 'Input1_2', 'Input1_3', 'Well_1', 'Well_2', 'Well_3', 'Chip_1', 'Chip_2', 'Chip_3'], axis = 1)
filtered_m['Count'] = (filtered_m[['Input2_1', 'Input2_2', 'Input2_3', 'Invivo_1', 'Invivo_2', 'Invivo_3']] == 0).sum(axis=1)
filtered_m = filtered_m[filtered_m['Count'] < 5]
filtered_m = filtered_m.drop(['Count'], axis = 1)

In [ ]:
#Make separated variables of dataframe with input background (transwell - tw, chip - c, in vivo - m)
m_tw = filtered_t.loc[:, ["Pep_seq", "Input1_1", "Input1_2", "Input1_3", "Well_1", "Well_2", "Well_3"]]
m_c = filtered_t.loc[:, ["Pep_seq", "Input1_1", "Input1_2", "Input1_3", "Chip_1", "Chip_2", "Chip_3"]]
m_m = filtered_m.loc[:, ["Pep_seq", "Input2_1", "Input2_2", "Input2_3", "Invivo_1", "Invivo_2", "Invivo_3"]]

In [ ]:
# Filtering the enriched peptide sequences using the criteria (top 2,000)
m_tw_top = m_tw.loc[(m_tw['Well_1'] <= m_tw['Well_2']) & (m_tw['Well_1'] < m_tw['Well_3']) & (m_tw['Input1_3'] < m_tw['Well_3']), :]
m_tw_top = m_tw_top.sort_values(by = ["Well_3"], axis = 0, ascending = False)
m_tw_top_input = m_tw_top.head(2000)
m_c_top = m_c.loc[(m_c['Chip_1'] <= m_c['Chip_2']) & (m_c['Chip_1'] < m_c['Chip_3']) & (m_c['Input1_3'] < m_c['Chip_3']), :]
m_c_top = m_c_top.sort_values(by = ["Chip_3"], axis = 0, ascending = False)
m_c_top_input = m_c_top.head(2000)
m_m_top = m_m.loc[(m_m['Invivo_1'] <= m_m['Invivo_2']) & (m_m['Invivo_1'] < m_m['Invivo_3']) & (m_m['Input2_3'] < m_m['Invivo_3']), :]
m_m_top = m_m_top.sort_values(by = ["Invivo_3"], axis = 0, ascending = False)
m_m_top_input = m_m_top.head(2000)

In [ ]:
# Filtering the not-enriched peptide sequences using the criteria (bottom 2,000)
m_tw_bot = m_tw.loc[(m_tw['Input1_1'] > m_tw['Input1_2']) & (m_tw['Input1_2'] >= m_tw['Input1_3']) & (m_tw['Input1_1'] > m_tw['Well_1']), :]
m_tw_bot = m_tw_bot.sort_values(by = ["Well_3"], axis = 0, ascending = True)
m_tw_bot_input = m_tw_bot.head(2000)
m_c_bot = m_c.loc[(m_c['Input1_1'] > m_c['Input1_2']) & (m_c['Input1_2'] >= m_c['Input1_3']) & (m_c['Input1_1'] > m_c['Chip_1']), :]
m_c_bot = m_c.sort_values(by = ["Chip_3"], axis = 0, ascending = True)
m_c_bot_input = m_c_bot.head(2000)
m_m_bot = m_m.loc[(m_m['Input2_1'] > m_m['Input2_2']) & (m_m['Input2_2'] >= m_m['Input2_3']) & (m_m['Input2_1'] > m_m['Invivo_1']), :]
m_m_bot = m_m.sort_values(by = ["Invivo_3"], axis = 0, ascending = True)
m_m_bot_input = m_m_bot.head(2000)

In [ ]:
#Add the class to each filtered sequence
#Enriched peptide : 1
#Not-enriched peptide : 0
m_tw_top_input.insert(7, 'Class', 1)
m_c_top_input.insert(7, 'Class', 1)
m_m_top_input.insert(7, 'Class', 1)
m_tw_bot_input.insert(7, 'Class', 0)
m_c_bot_input.insert(7, 'Class', 0)
m_m_bot_input.insert(7, 'Class', 0)

In [ ]:
#Concatenate the enriched and not-enriched peptides in one dataframe
merged_input_tw = pd.concat([m_tw_top_input, m_tw_bot_input], axis = 0)
merged_input_c = pd.concat([m_c_top_input, m_c_bot_input], axis = 0)
merged_input_m = pd.concat([m_m_top_input, m_m_bot_input], axis = 0)

In [ ]:
#Export the dataframe in each experiment (transwell, chip, in vivo) that is applied to machine learning prediction (ML)
merged_input_tw.to_csv("INPUT_FILE_PATH_FOR_ML/Input4000_ML_tw.csv", sep = "\t")
merged_input_c.to_csv("INPUT_FILE_PATH_FOR_ML/Input4000_ML_c.csv", sep = "\t")
merged_input_m.to_csv("INPUT_FILE_PATH_FOR_ML/Input4000_ML_m.csv", sep = "\t")